In [1]:
import psycopg2
import json
import ipyleaflet as L
import networkx as nx
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

## Database connection

In [2]:
def connect(broken):# Set connection to gis db
    if broken:
        conn.close()
    try:
        conn = psycopg2.connect("dbname=gis user=jennyzhou")
    except:
        print 'Fail to connect to Postgres Server'
    return conn
        
conn = connect(False)

## Load Result from database

In [3]:
cur = conn.cursor()
# Select one polygon area
vid = 38
types = ['MSA','MST','NS']
graphJson = {}
# Select vornoi polygon and transformer
cur.execute("SELECT ST_AsGeoJSON((ST_FlipCoordinates(ST_Transform(way, 4326)))), "
            "ST_AsGeoJSON((ST_FlipCoordinates(ST_Transform(tway, 4326)))) FROM vornoi WHERE vid = %d ;" % vid)
data = cur.fetchall()
vornoiJson = json.loads(data[0][0])
transJson = json.loads(data[0][1])

# Select all the houses
cur.execute("SELECT ST_AsGeoJSON(ST_Collect(ST_FlipCoordinates(ST_Transform(way, 4326)))) "
            "FROM vornoi_map where vid = %d and istransformer = false ;" % vid)
houses = cur.fetchall()
houseJson = json.loads(houses[0][0])

for name in types:
    # Get result of Minimun Spanning Tree
    cur.execute("SELECT ST_AsGeoJSON(ST_FlipCoordinates(ST_Transform(way, 4326)))"
                "FROM graph WHERE type = '%s' AND vid = %d;" %(name,vid))
    lines = cur.fetchall()
    graphJson[name] = json.loads(lines[0][0])
cur.close()

## Visualisation on map

In [4]:
# Modulize retrival of map for different visualization simultaneously
def getMap(center, boundary):
    # Mark transformer
    tMarker =  Marker(location=center['coordinates'])
    # Mark polygon
    tPoly = Polygon(locations=boundary['coordinates'], weight=2,
                color='#a3c2c2', opacity=0.8, fill_opacity=0.2,
                fill_color='#ccffcc') 

    # Init Map
    tMap = Map(default_tiles=TileLayer(opacity=1.0),center=center['coordinates'] ,zoom=17)
    tMap.add_layer(tMarker)
    tMap.add_layer(tPoly)
    return tMap

def markHouse(tMap, data):
    markers = []
    # Mark houses
    for hp in data['coordinates']:
        c = Circle(location= hp, weight=5, opacity = 0.5, color = 'red', radius = 2) 
        markers.append(c)
        tMap.add_layer(c)
    return markers

def markGraph(tMap, data):
    markers = []
    for l in data['coordinates']:
        pl = Polyline(locations=l, weight=2, color='#2929a3', opacity=0.5)
        markers.append(pl)
        tMap.add_layer(pl)
    return markers
    
def cleanMarker(tMap, markers):
    if len(markers) > 0:
        for mk in markers:
            tMap.remove_layer(mk)

In [5]:
# Test
tMap = getMap(transJson, vornoiJson)
hMarkers = markHouse(tMap, houseJson)
tMap

In [20]:
# Remove house markers
cleanMarker(tMap, hMarkers)

## Minimum Spanning Tree

In [6]:
mstMap = getMap(transJson, vornoiJson)
#hMarkers = markHouse(tMap, houseJson)
mstMarkers = markGraph(mstMap, graphJson['MST'])
mstMap

In [9]:
cleanMarker(mstMap, mstMarkers)

## Minimum Spanning Arborescence

In [6]:
msaMap = getMap(transJson, vornoiJson)
msaMarkers = markGraph(msaMap, graphJson['MSA'])
msaMap

In [ ]:
cleanMarker(msaMap, msaMarkers)

## Network Simplex

In [7]:
nsMap = getMap(transJson, vornoiJson)
nsMarkers = markGraph(nsMap, graphJson['NS'])
nsMap

In [ ]:
cleanMarker(nsMap, nsMarkers)